# Current version : 6.10.a (2024-11-13)

# Libraries and directory (always run)

In [3]:
### import necessary libraries
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import scanpy as sc
from scipy.optimize import curve_fit
import warnings

warnings.filterwarnings("ignore") 
sc.logging.print_header()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 1 # errors (0), warnings (1), info (2), hints (3)
plt.rcParams["font.family"] = "Arial"
sns.set_style("white")

# Note that BANKSY itself is deterministic, here the seeds affect the umap clusters and leiden partition
seed = 1234
np.random.seed(seed)
random.seed(seed)

start_time = datetime.now()

def print_with_elapsed_time(message):
    elapsed_time = datetime.now() - start_time
    elapsed_seconds = elapsed_time.total_seconds()
    print(f"[{elapsed_seconds:.2f} seconds] {message}")

In [4]:
print(f"geopandas version: {gpd.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"scanpy version: {sc.__version__}")

geopandas version: 1.0.1
pandas version: 2.0.1
scanpy version: 1.11.1


In [ ]:
### Directory where the data is stored

# dir = "/mnt/d/Xenium" #Ubuntu
# dir = "/media/volume/data/spatial/hugo/data" #Ubuntu
dir = "/media/volume/data/spatial/hugo/data/k5" #Ubuntu
dir = '/media/volume/volume_spatial/hugo/data/test'
dir = '/media/volume/volume_spatial/hugo/data'

# dir_notebook = '/mnt/d/Jupyter_notebook/Xenium_jupyter_notebook'
# dir_notebook = '/media/volume/data/spatial/hugo/notebook'
dir_notebook = '/media/volume/volume_spatial/hugo/notebook'


In [ ]:
# # # # ### HC ###

# # # # ### make a list of samples and their ids to make the cell names unique
# # # # #samples = ["2505-1__20240404__003359", "2505-2__20240404__003359", "2670-1__20240404__003359", "3159-1__20240321__212855", "3160-1__20240321__212855", "3160-2__20240321__212855"]
# # # # #samples_ids = ["2505-1", "2505-2", "2670-1", "3159-1", "3160-1", "3160-2"]
# # # # #name_dir = 'run1-2'

# # # # # ### Run 1 resegmented
# # # # samples = ["2505-1_subsampled", "2505-2_subsampled", "2670-1_subsampled", 
# # # #    "3159-1_subsampled"
# # # #     , "3160-1_subsampled", "3160-2_subsampled"]
# # # # samples_ids = ["2505-1", "2505-2", "2670-1", 
# # # #               "3159-1", "3160-1", "3160-2"       ]
# # # # name_dir = 'run1-resegment'

# # # # # # #Run 3
# # # # # samples = ["3159-2__20240530__205547", "3159-3__20240530__205547", "3159-4__20240530__205547", "3161-1__20240530__205547", "3161-2__20240530__205547", "3161-3__20240530__205547"]
# # # # # samples_ids = ["3159-2", "3159-3", "3159-4", "3161-1", "3161-2", "3161-3"]
# # # # # name_dir = 'run3-all'

# # # # # Run 3 Habenula
# # # samples = ["3159-2__20240530__205547", "3161-1__20240530__205547"]
# # # samples_ids = ["3159-2", "3161-1"]
# # # name_dir = 'run3-Habenula'

# # # # # # # Run 3 LGN
# # # # samples = ["3159-3__20240530__205547", "3161-2__20240530__205547"]
# # # # samples_ids = ["3159-3","3161-2"]
# # # # name_dir = 'run3-LGN'

# # # # # #Run 3 SC
# # # samples = ["3159-4__20240530__205547","3161-3__20240530__205547"]
# # # samples_ids = ["3159-4", "3161-3"]
# # # name_dir = 'run3-SC'

# # # # # samples = ["Xenium_Prime_Mouse_Brain_Coronal_FF_outs"]
# # # # # samples_ids = ['Xenium_Prime']
# # # # # name_dir = 'test-prime'

# # # # # # circa1
# # samples = ['circa1_ZT1','circa1_ZT5','circa1_ZT9','circa1_ZT13','circa1_ZT17','circa1_ZT21']
# # samples_ids = ['circa1-ZT1','circa1-ZT5','circa1-ZT9','circa1-ZT13','circa1-ZT17','circa1-ZT21',]
# # name_dir = 'circa1'

# # # ### circa2_
# samples = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21']
# samples_ids = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21',]
# name_dir = 'circa2'

# # ### circa3_
# samples = ['circa3-ZT01','circa3-ZT05','circa3-ZT09','circa3-ZT13','circa3-ZT17','circa3-ZT21']
# samples_ids = ['circa3-ZT01','circa3-ZT05','circa3-ZT09','circa3-ZT13','circa3-ZT17','circa3-ZT21',]
# name_dir = 'circa3'

# ### circa4-Salk
# samples = ['circa4-Salk-ZT01',
#            'circa4-Salk-ZT05','circa4-Salk-ZT09','circa4-Salk-ZT13','circa4-Salk-ZT17',
#            'circa4-Salk-ZT21',]
# samples_ids = ['circa4-Salk-ZT01',
#                'circa4-Salk-ZT05','circa4-Salk-ZT09','circa4-Salk-ZT13','circa4-Salk-ZT17',
#                'circa4-Salk-ZT21',]
# name_dir = 'circa4-Salk'


# circa4-IGM
# samples_ids = ['circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09'
#                    ,'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
#                    ]
# samples = ['circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09'
#                    ,'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
#                    ]
# name_dir = 'circa4-IGM'


### All 5k
samples_ids = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21',
            'circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09',
            'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
            'circa4-Salk-ZT01','circa4-Salk-ZT05','circa4-Salk-ZT09',
            'circa4-Salk-ZT13','circa4-Salk-ZT17','circa4-Salk-ZT21',
                ]
samples = ['circa2-ZT01','circa2-ZT05','circa2-ZT09','circa2-ZT13','circa2-ZT17','circa2-ZT21',
            'circa4-IGM-ZT01','circa4-IGM-ZT05','circa4-IGM-ZT09',
            'circa4-IGM-ZT13','circa4-IGM-ZT17','circa4-IGM-ZT21',
            'circa4-Salk-ZT01','circa4-Salk-ZT05','circa4-Salk-ZT09',
            'circa4-Salk-ZT13','circa4-Salk-ZT17','circa4-Salk-ZT21',
               ]
name_dir = 'circa'


# # # # # # ### ALL SAMPLES
# samples = ["2505-1__20240404__003359", "2505-2__20240404__003359", "2670-1__20240404__003359", "3159-1__20240321__212855",
#            "3160-1__20240321__212855", "3160-2__20240321__212855","3159-2__20240530__205547", "3161-1__20240530__205547",
#           "3159-3__20240530__205547", "3161-2__20240530__205547","3159-4__20240530__205547","3161-3__20240530__205547"]

# samples_ids = ["2505-1", "2505-2", "2670-1", "3159-1",
#                "3160-1", "3160-2","3159-2", "3161-1",
#               "3159-3","3161-2","3159-4", "3161-3"
#               ]
# name_dir = 'all-samples'

# samples = ["CONTROLS","GCA"]
# samples_ids = ["CONTROLS","GCA"]
# name_dir = "Human-arteries"

# samples = ["Region1"
#         #    ,"Region2","Region3"
#            ]
# samples_ids = ["Region1"
#             #    ,"Region2","Region3"
#                ]
# name_dir = "MB_test"


# samples = ['S1_Region1']
# samples_ids = ["S1_Region1"]
# name_dir = 'march-test'

# Analysis

## Data Import (always run)

## Find marker genes for each cluster

In [ ]:
# Obtain cluster-specific differentially expressed genes
# cluster_to_use = 'cell_type_newnum'
# cluster_to_use = 'cell type'
# cluster_to_use = 'cell_type_auto'
# cluster_to_use = 'cell_type_auto_sub'
# cluster_to_use = 'sample'
cluster_to_use = 'cell_type_final'
# sc.pp.normalize_total(adata, inplace=True)
# sc.pp.log1p(adata)
adata.obs[cluster_to_use] = adata.obs[cluster_to_use].astype(str)
sc.tl.rank_genes_groups(adata, groupby=cluster_to_use, method="wilcoxon", tie_correct = True)


In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby=cluster_to_use, standard_scale="var", n_genes=1, dendrogram = True)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    n_genes=1,
    values_to_plot="logfoldchanges", cmap='bwr',
    vmin=-4,
    vmax=4,
)

In [ ]:
dat = pd.DataFrame()
for i in range(0, len(adata.obs[cluster_to_use].unique())):
    print(f"Cluster {i}")
    dat1 = sc.get.rank_genes_groups_df(adata, group=str(i))
    dat1['group'] = i
    dat = pd.concat([dat, dat1])

dat.to_csv("marker_genes_0-4_run1.csv")

In [ ]:
marker_genes = [
# 10X annotations
# "Acsbg1","Aqp4","Cdh20","Clmn","Gfap","Gli3","Id2","Mapk4","Ntsr2","Pde7b","Rfx4","Rorb","Slc39a12", #Astrocytes
# "Arhgap12","Fibcd1","Sipa1l3","Wfs1", #CA1-ProS
# "2010300C02Rik","Arhgef28","Bcl11b","Bhlhe22","Cabp7","Cpne4","Igfbp4","Necab2","Prdm8","Strip2","Syndig1", #CA2
# "Cpne6","Epha4","Hat1","Neurod6","Npy2r","Nrp2","Shisa6", #CA3
# "Cdh9","Orai2","Prox1","Rasl10a","Tanc1", #DG
# "Acvrl1","Adgrl4","Car4","Cd93","Cldn5","Cobll1","Emcn","Fgd5","Fn1","Kdr","Ly6a","Mecom","Nostrin","Paqr5","Pecam1","Pglyrp1","Slfn5","Sox17","Zfp366", #Endothelial
#"Arhgap25","Cd300c2","Cd53","Cd68","Ikzf1","Laptm5","Siglech","Sla","Spi1","Trem2",#Microglia
# 'Gjc3','Gpr17','Opalin','Sema3d','Sema6a','Sox10','Zfp536', #Oligodendrocytes
# 'Acta2','Ano1','Arhgap6','Carmn','Cspg4','Fos','Gucy1a1','Inpp4b','Nr2f2','Pip5k1b','Plekha2','Pln','Sncg','Sntb1', # Pericytes
# 'Aldh1a2','Col1a1','Col6a1','Cyp1b1','Dcn','Fmod','Gjb2','Igf2','Pdgfra','Ror1',"Slc13a4","Spp1", #VLMC
# "Chat","Crh","Igf1",'Penk','Pthlh','Sorcs3','Thsd7a','Vip', #Vip interneurons

'Arntl','Clock','Cry1','Cry2','Nr1d1',"Per1",'Per2','Per3','Rora','Rorb', ## Clock genes
# 'Gfap','Trem2','Cd44','Spp1','Cd68','Igf1','Spi1','Cd300c2','Cd53','Laptm5','Ikzf1','Arhgap25','Opalin','Prox1','Cbln1','Sema3a','Paqr5','Spag16',
# "Vip", "Pkib", "Tmem255a", "Arhgap6", "Chodl", #SCN rank genes analysis
# 'Strip2',"Shisa6","Chodl", 'Fos','Sdk2', 'Cdh6','Cobll1','Tanc1'
# 'Dner','Gad1','Rasgrf2','Vat1l','Pde7b','Igfbp5','Rorb','Rims3','Tmem255a','Cdh13','Gad2','Rab3b','Parm1','Tle4','Fhod3','Rmst','Vip','Nr2f2','Arhgap6',
# 'Laptm5','Kctd12','Siglech','Trem2','Cd53','Cd68','Cd300c2','Ikzf1','Spi1','Acsbg1','Gfap','Dpy19l1','Unc13c','Arhgap25','Meis2','Dner','Arhgap12','Igfbp5','Ntsr2',
# "Gfap","Rbp4","Trem2","Th","Laptm5","Syt17","Opn3","Spp1","Cd44","Cd53","Igf1","Gjb2",
# "Vip","Prss35","Cd68","Cplx3","Siglech","Ikzf1","Cd300c2","Dcn","Spi1","Pkib","Fos","Angpt1",
# "Igfbp5","Chrm2","Rspo2","Arhgap25","Sst",
# 'Ntsr2'    
]


In [ ]:
ax = sc.pl.stacked_violin(adata2,
                         marker_genes,
                         groupby='cell_type_final',
                         dendrogram=False,
                         log=False,
                         )

In [ ]:
sc.pl.violin(adata2, marker_genes, groupby='Genotype', order = ['WT','APP'],
             jitter = 0.45,
             # log = True,
             # stripplot = False,
            )

## Subset (a)dataset for one cell type

In [ ]:
adata.obs.groupby('cell_type_final')['cell_id'].count()

In [ ]:
celltype_to_subset = "SCH Gaba"
adata_microglia = adata[adata.obs['cell_type_final'] == celltype_to_subset]

In [ ]:
adata_microglia.obs['sample']#,adata_microglia.obs['Genotype']

In [ ]:
marker_genes = [
# 10X annotations
# "Acsbg1","Aqp4","Cdh20","Clmn","Gfap","Gli3","Id2","Mapk4","Ntsr2","Pde7b","Rfx4","Rorb","Slc39a12", #Astrocytes
# "Arhgap12","Fibcd1","Sipa1l3","Wfs1", #CA1-ProS
# "2010300C02Rik","Arhgef28","Bcl11b","Bhlhe22","Cabp7","Cpne4","Igfbp4","Necab2","Prdm8","Strip2","Syndig1", #CA2
# "Cpne6","Epha4","Hat1","Neurod6","Npy2r","Nrp2","Shisa6", #CA3
# "Cdh9","Orai2","Prox1","Rasl10a","Tanc1", #DG
# "Acvrl1","Adgrl4","Car4","Cd93","Cldn5","Cobll1","Emcn","Fgd5","Fn1","Kdr","Ly6a","Mecom","Nostrin","Paqr5","Pecam1","Pglyrp1","Slfn5","Sox17","Zfp366", #Endothelial
# "Arhgap25","Cd300c2","Cd53","Cd68","Ikzf1","Laptm5","Siglech","Sla","Spi1","Trem2",#Microglia
# 'Gjc3','Gpr17','Opalin','Sema3d','Sema6a','Sox10','Zfp536', #Oligodendrocytes
# 'Acta2','Ano1','Arhgap6','Carmn','Cspg4','Fos','Gucy1a1','Inpp4b','Nr2f2','Pip5k1b','Plekha2','Pln','Sncg','Sntb1', # Pericytes
# 'Aldh1a2','Col1a1','Col6a1','Cyp1b1','Dcn','Fmod','Gjb2','Igf2','Pdgfra','Ror1',"Slc13a4","Spp1", #VLMC
# "Chat","Crh","Igf1",'Penk','Pthlh','Sorcs3','Thsd7a','Vip', #Vip interneurons

'Arntl','Clock','Cry1','Cry2','Nr1d1',"Per1",'Per2','Per3','Rora','Rorb', ## Clock genes

# "Vip", "Pkib", "Tmem255a", "Arhgap6", "Chodl", #SCN rank genes analysis
# 'Strip2',"Shisa6","Chodl", 'Fos','Sdk2', 'Cdh6','Cobll1','Tanc1'
# 'Dner','Gad1','Rasgrf2','Vat1l','Pde7b','Igfbp5','Rorb','Rims3','Tmem255a','Cdh13','Gad2','Rab3b','Parm1','Tle4','Fhod3','Rmst','Vip','Nr2f2','Arhgap6',
# 'Laptm5','Kctd12','Siglech','Trem2','Cd53','Cd68','Cd300c2','Ikzf1','Spi1','Acsbg1','Gfap','Dpy19l1','Unc13c','Arhgap25','Meis2','Dner','Arhgap12','Igfbp5','Ntsr2',
# "Gfap","Rbp4","Trem2","Th","Laptm5","Syt17","Opn3","Spp1","Cd44","Cd53","Igf1","Gjb2",
# "Vip","Prss35","Cd68","Cplx3","Siglech","Ikzf1","Cd300c2","Dcn","Spi1","Pkib","Fos","Angpt1",
# "Igfbp5","Chrm2","Rspo2","Arhgap25","Sst",
# 'Ntsr2'    
]

In [ ]:
sc.pl.stacked_violin(adata_microglia, marker_genes, groupby='sample', dendrogram=False,)

In [ ]:
sc.pl.violin(adata_microglia, marker_genes, groupby='ZT',
             #order = ['WT','APP'],
             #log = True,
             # stripplot = False,
            )

In [ ]:
    sc.tl.rank_genes_groups(adata_microglia, groupby=cluster_to_use, method="wilcoxon", tie_correct = True, pts = True)
    
    # sc.pl.rank_genes_groups_dotplot(adata_microglia, groupby=cluster_to_use, standard_scale="var", n_genes=5)
    
    for i in adata2.obs[cluster_to_use].unique():
        # print(f"Cluster {cell_type_to_extract}_{i}")
        dat1 = sc.get.rank_genes_groups_df(adata_microglia, group=i)
        dat1['group'] = i
        dat = pd.concat([dat, dat1])

    dat = dat[ ### Choose filters here
    (dat['pct_nz_group'] > 0.15) & #Percentage of cell expressing the gene
    (dat['pvals_adj']<= 0.05) & # adjusted p-value
    (abs(dat['logfoldchanges']) > 0.26) # logfoldchange
    ]

In [ ]:
cluster_to_use = "ZT"

sc.tl.rank_genes_groups(adata_microglia, groupby=cluster_to_use, method="wilcoxon", tie_correct = True, pts = True)


dat = pd.DataFrame()
for i in adata_microglia.obs[cluster_to_use].unique():
    print(f"Cluster {i}")
    dat1 = sc.get.rank_genes_groups_df(adata_microglia, group=str(i))
    dat1['group'] = i
    dat = pd.concat([dat, dat1])

dat = dat[ ### Choose filters here
    (dat['pct_nz_group'] > 0.15)  #Percentage of cell expressing the gene
    & (dat['pvals_adj']<= 0.05) # # adjusted p-value
    # & (abs(dat['logfoldchanges']) > 0.26) # logfoldchange
    ]

dat.to_csv("data/SCH.csv")

In [ ]:
sc.pl.highest_expr_genes(adata_microglia, n_top=30, show=None, save=None, ax=None, gene_symbols=None, log=False)

In [ ]:
adata_microglia_WT=adata_microglia[adata_microglia.obs['Genotype']== 'WT']
adata_microglia_APP=adata_microglia[adata_microglia.obs['Genotype']== 'APP']

sc.pl.highest_expr_genes(adata_microglia_WT, n_top=30, show=None, save=None, ax=None, gene_symbols=None, log=False)
sc.pl.highest_expr_genes(adata_microglia_APP, n_top=30, show=None, save=None, ax=None, gene_symbols=None, log=False)

### Subcluster the subset

In [ ]:
# extract pca coordinates
# X_pca = adata_microglia.obsm['X_pca'] 

sc.pp.pca(adata_microglia)
sc.pp.neighbors(adata_microglia)
sc.tl.umap(adata_microglia)

### Kmeans clustering
# ### You can choose the number of clusters by uncommenting n_clusters option
# kmeans = KMeans(#n_clusters=2,
#                 random_state=0).fit(X_pca) 
# adata_microglia.obs['kmeans'] = kmeans.labels_.astype(str)

sc.tl.leiden(adata_microglia, resolution = 0.2)

In [ ]:
# adata.obs.groupby('cell type')[['nucleus_area','cell_area','total_counts','n_genes_by_counts']].max().sort_values(by = 'cell_area')

In [ ]:
### Choose one cluster to work with
#cluster_to_use = 'leiden0.4_classname'
# cluster_to_use = 'kmeans'
cluster_to_use = 'leiden'
# cluster_to_use = 'L04_newnum'

In [ ]:
### Number of cells per clusters
max_clust = len(adata_microglia.obs[cluster_to_use].unique())
for i in range(0, max_clust):
    count = adata_microglia.obs[cluster_to_use].value_counts().iloc[i]
    print(f"Cluster {i} : {count} cells")

In [ ]:
### Generate a color palette for the clusters - to make color stay consistent across samples
adata_microglia.obs[cluster_to_use] = adata_microglia.obs[cluster_to_use].astype(str)

# Create a palette with a unique color for each cluster
num_clusters = len(adata_microglia.obs[cluster_to_use].astype(int).unique())
palette = sns.color_palette("tab20", n_colors=num_clusters)

# Map each 'leiden' value to a color
adata_microglia.obs['leiden_colors'] = adata_microglia.obs[cluster_to_use].astype(int).apply(lambda x: palette[x])

In [ ]:
### Let's make UMAP plot. We will also add the cluster centroids to the plot
adata_microglia.obs['umap-1'] = adata_microglia.obsm['X_umap'][:, 0]
adata_microglia.obs['umap-2'] = adata_microglia.obsm['X_umap'][:, 1]
cluster_centroids = adata_microglia.obs.groupby(cluster_to_use)[['umap-1', 'umap-2']].median()

In [ ]:
## Draw UMAP
fig, axs = plt.subplots(2,4,figsize=(30, 15))
axs = axs.flatten()

def plot_umap(adata_microglia, color_column, ax, title=None):
    scatter = ax.scatter(adata_microglia.obsm['X_umap'][:, 0], adata_microglia.obsm['X_umap'][:, 1], c=adata_microglia.obs[color_column], s=2, alpha=0.8)
    ax.set_title(title)
    ax.axis('off')

for i, sample in enumerate(samples_ids):
    sample_data = adata_microglia[adata_microglia.obs['sample'] == sample]
    plot_umap(sample_data, 'leiden_colors', axs[i], title=f"UMAP for {sample}")
    
    for cluster_id, centroid in cluster_centroids.iterrows():
        axs[i].text(centroid['umap-1'], centroid['umap-2'], str(cluster_id), color='black', fontsize=9)

# plt.savefig(f"/mnt/d/Jupyter_notebook/Xenium_jupyter_notebook/plot/{name_dir}/{name_dir}_UMAP_{cluster_to_use}.svg")

In [ ]:
samples_ids = adata_microglia.obs['sample'].unique()

In [ ]:
# Map all cells
fig, axs = plt.subplots(2,4,figsize=(30, 15))
axs = axs.flatten()
clusters_plot = {"1": 'magenta',"0": 'cyan', "2": 'green', "":'red', "":'orange',"7":'black',"":"purple"
                }

for idx, sample in enumerate(samples_ids):
    adata_sel = adata_microglia[(adata_microglia.obs['sample'] == sample)]
    for cluster_id in adata_sel.obs[cluster_to_use].unique():
        cluster_data = adata_sel.obs[adata_sel.obs[cluster_to_use] == cluster_id]
        colors = clusters_plot[cluster_id] if cluster_id in clusters_plot else "none" ### for selected clusters in cluster_plot
        # colors= cluster_data['leiden_colors'].unique()[0] ### for all clusters
        axs[idx].scatter(cluster_data['x_centroid'].astype('float'), cluster_data['y_centroid'].astype('float'), color=colors, s=10, label=cluster_id)
        axs[idx].set_title(f"Sample {sample}")
        # axs[idx].set_ylim(400,1300)
        # axs[idx].set_xlim(4100,5600)



In [ ]:
### Correlation map of subclusters
cont_tab = pd.crosstab(adata_microglia.obs[cluster_to_use], adata_microglia.obs['mmc:supertype_name'], normalize="index")
plt.figure(figsize=(20, 10))
sns.heatmap(cont_tab, annot=True, cmap="YlGnBu", fmt=".1f")

In [ ]:
# Obtain cluster-specific differentially expressed genes
# cluster_to_use = 'leid'
# cluster_to_use = 'Genotype'
adata_microglia.obs[cluster_to_use] = adata_microglia.obs[cluster_to_use].astype(str)
# sc.tl.dendrogram(adata_microglia, groupby = cluster_to_use, n_pcs=None, use_rep=None, var_names=None, use_raw=None, cor_method='pearson', linkage_method='complete', optimal_ordering=False, key_added=None)
sc.tl.rank_genes_groups(adata_microglia, groupby=cluster_to_use, method="wilcoxon")
sc.pl.rank_genes_groups_dotplot(adata_microglia, groupby=cluster_to_use, standard_scale="var", n_genes=5)

sc.pl.rank_genes_groups_dotplot(
    adata_microglia,
    n_genes=5,
    values_to_plot="logfoldchanges", cmap='bwr',
    vmin=-4,
    vmax=4,
)

In [ ]:
sc.pl.rank_genes_groups_violin(adata_microglia, groups=adata_microglia.obs[cluster_to_use], n_genes=1)

In [ ]:
marker_genes = ['Gfap', "Igf1", "Gng12", "Cd68", "Igfbp5", "Siglech", "Ikzf1", "Kctd12", "Cd300c2", "Spi1"]

In [ ]:
ax = sc.pl.stacked_violin(adata_microglia, var_names = marker_genes , groupby='kmeans', dendrogram=True)

In [ ]:
# Obtain cluster-specific differentially expressed genes
# cluster_to_use = 'kmeans'
cluster_to_use = 'ZT'
adata_microglia.obs[cluster_to_use] = adata_microglia.obs[cluster_to_use].astype(str)
# sc.tl.dendrogram(adata_microglia, groupby = cluster_to_use, n_pcs=None, use_rep=None, var_names=None, use_raw=None, cor_method='pearson', linkage_method='complete', optimal_ordering=False, key_added=None)
sc.tl.rank_genes_groups(adata_microglia, groupby=cluster_to_use, method="wilcoxon", tie_correct = True)
sc.pl.rank_genes_groups_dotplot(adata_microglia, groupby=cluster_to_use, standard_scale="var", n_genes=5, dendrogram = False)

sc.pl.rank_genes_groups_dotplot(
    adata_microglia,
    n_genes=5,
    values_to_plot="logfoldchanges", cmap='bwr',
    dendrogram = False
    # vmin=-4,
    # vmax=4,
)

In [ ]:
### Extract gene expression per cluster + log fold change + p-value
cluster_to_use = 'Genotype'

adata_microglia.obs[cluster_to_use] = adata_microglia.obs[cluster_to_use].astype(str)
#sc.tl.dendrogram(adata, groupby = 'L04_newnum_subclassname')
sc.tl.rank_genes_groups(adata_microglia, groupby=cluster_to_use, method="wilcoxon", tie_correct = True, pts = True)

sc.pl.rank_genes_groups_dotplot(adata_microglia, groupby=cluster_to_use, standard_scale="var", n_genes=5)

dat = pd.DataFrame()
for i in adata2.obs[cluster_to_use].unique():
    print(f"Cluster {i}")
    dat1 = sc.get.rank_genes_groups_df(adata_microglia, group=i)
    dat1['group'] = i
    dat = pd.concat([dat, dat1])

### Extract not-normalized expression and clusters for individual cells
if not os.path.exists(f"{dir_notebook}/csv/{name_dir}/"):
   os.makedirs(f"{dir_notebook}/csv/{name_dir}/")
dat.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_clusters_foldchange_{celltype_to_subset}.csv")

In [ ]:
dat

In [ ]:
celltype_to_subset = "SCH"
adata_region_cell = adata2[adata2.obs['cell type'] == celltype_to_subset]

## Subset one region

In [ ]:
adata2.obs['region_automap'].unique()

In [ ]:
region_to_subset = "SCH"
adata_region = adata2[adata2.obs['region_automap'] == region_to_subset]

In [ ]:
adata_region.obs[adata_region.obs['cell_type_final']== 'SCH Gaba'].groupby('mmc:subclass_name')['cell_id'].nunique()

In [ ]:
adata_region.obs[adata_region.obs['cell_type_final']=='SCH']['cell_type_newnum_final'].head()

In [ ]:
# Generate new numbering base on unique 'cell type'
all_cell_type = adata_region.obs['cell type'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata_region.obs['cell_type_newnum'] = adata_region.obs['cell type'].map(mapping_dict)
# mapping_dict

### Generate a color palette for the clusters - to make color stay consistent across samples
adata_region.obs['cell_type_newnum'] = adata_region.obs['cell_type_newnum'].astype(int)

# Create a palette with a unique color for each cluster
num_clusters = len(adata_region.obs['cell_type_newnum'].unique())
palette = sns.color_palette("bright", n_colors=num_clusters)

# Map each 'leiden' value to a color
adata_region.obs['kmeans_colors'] = adata_region.obs['cell_type_newnum'].apply(lambda x: palette[x])

# Mapping of clusters
fig, axs = plt.subplots(3,2,figsize=(15, 18))
axs = axs.flatten()
clusters_plot = { 1: 'lightcoral', 11:'black',4:'red',
    # 0: 'orchid', 1: 'forestgreen',2: 'coral', 4:'orange',
    # 3:'red', 5:'blue',6:'cyan',7:'black'
    # 4:'red',0:'black'
}

for idx, sample in enumerate(samples_ids):
    adata_sel = adata_region[(adata_region.obs['sample'] == sample)]
    for cluster_id in adata_sel.obs['cell_type_newnum'].unique():
        cluster_data = adata_sel.obs[adata_sel.obs['cell_type_newnum'] == cluster_id]
        if len(cluster_data) >= 0:
            colors = clusters_plot[cluster_id] if cluster_id in clusters_plot else "none"
            colors= cluster_data['kmeans_colors'].unique()[0]
            axs[idx].scatter(cluster_data['x_centroid'], cluster_data['y_centroid'], color=colors, s=15, label=cluster_data['cell type'].unique()[0])
            axs[idx].set_title(f"Sample {sample}")

plt.legend(markerscale=1, scatterpoints=1000, bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)


In [ ]:
# len(adata_region)/1000
# adata_region_notunique.obs.groupby('cell type')['cell_id'].nunique()
# list_to_exclude

In [ ]:
list_to_exclude  = adata_region.obs.groupby('cell type')['cell_id'].nunique() >= 10
list_to_exclude.values
dict_exclude = dict(zip(list_to_exclude.index, list_to_exclude.values))
dict_exclude

adata_region.obs['exclude'] = adata_region.obs['cell type'].map(dict_exclude)

adata_region_notunique = adata_region[adata_region.obs['exclude'] != False]

adata_region_notunique.obs.groupby('cell type')['cell_id'].nunique()

sc.tl.dendrogram(adata_region_notunique, groupby = 'cell type', n_pcs=None, use_rep=None, var_names=None, use_raw=None, cor_method='pearson', linkage_method='complete', optimal_ordering=False, key_added=None)
sc.tl.rank_genes_groups(adata_region_notunique, groupby='cell type', method="wilcoxon", tie_correct = True)
sc.pl.rank_genes_groups_dotplot(adata_region_notunique, groupby='cell type', standard_scale="var", n_genes=5)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata_region_notunique,
    n_genes=5,
    values_to_plot="logfoldchanges", cmap='bwr',
    # vmin=-4,
    # vmax=4,
)

In [ ]:
### Extract gene expression per cluster + log fold change + p-value
cluster_to_use = 'Genotype'

adata_region.obs[cluster_to_use] = adata_region.obs[cluster_to_use].astype(str)
#sc.tl.dendrogram(adata, groupby = 'L04_newnum_subclassname')
sc.tl.rank_genes_groups(adata_region, groupby=cluster_to_use, method="wilcoxon", tie_correct = True, pts = True)

sc.pl.rank_genes_groups_dotplot(adata_region, groupby=cluster_to_use, standard_scale="var", n_genes=5)

dat = pd.DataFrame()
for i in adata_region.obs[cluster_to_use].unique():
    print(f"Cluster {i}")
    dat1 = sc.get.rank_genes_groups_df(adata_region, group=i)
    dat1['group'] = i
    dat = pd.concat([dat, dat1])

### Extract not-normalized expression and clusters for individual cells
if not os.path.exists(f"{dir_notebook}/csv/{name_dir}/"):
   os.makedirs(f"{dir_notebook}/csv/{name_dir}/")
dat.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_clusters_foldchange_{region_to_subset}-region.csv")

In [ ]:
celltype_to_subset = "Astro"
adata_region_cell = adata_region[adata_region.obs['cell type'] == celltype_to_subset]

In [ ]:
### Extract gene expression per cluster + log fold change + p-value
cluster_to_use = 'Genotype'

adata_region_cell.obs[cluster_to_use] = adata_region_cell.obs[cluster_to_use].astype(str)
#sc.tl.dendrogram(adata, groupby = 'L04_newnum_subclassname')
sc.tl.rank_genes_groups(adata_region_cell, groupby=cluster_to_use, method="wilcoxon", tie_correct = True, pts = True)

sc.pl.rank_genes_groups_dotplot(adata_region_cell, groupby=cluster_to_use, standard_scale="var", n_genes=5)

dat = pd.DataFrame()
for i in adata_region_cell.obs[cluster_to_use].unique():
    print(f"Cluster {i}")
    dat1 = sc.get.rank_genes_groups_df(adata_region_cell, group=i)
    dat1['group'] = i
    dat = pd.concat([dat, dat1])

### Extract not-normalized expression and clusters for individual cells
if not os.path.exists(f"{dir_notebook}/csv/{name_dir}/"):
   os.makedirs(f"{dir_notebook}/csv/{name_dir}/")
dat.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_clusters_foldchange_{region_to_subset}-region-{celltype_to_subset}-cell.csv")

# Output files

## Extract foldchange, pvalue for celltypes

In [ ]:
# adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz")

In [ ]:
adata2 = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_norm.h5ad.gz")

In [ ]:
mapping_dict_region = dict(zip(adata.obs['cell_id'], adata.obs['region_automap_name']))
mapping_dict_celltype = dict(zip(adata.obs['cell_id'], adata.obs['cell_type_final']))
mapping_dict_celltype_num = dict(zip(adata.obs['cell_id'], adata.obs['cell_type_newnum_final']))
# mapping_dict_manos = dict(zip(adata.obs['cell_id'], adata.obs['sample']))

adata2.obs['cell_type_newnum_final'] = adata2.obs['cell_id'].map(mapping_dict_celltype_num)
adata2.obs['cell_type_final'] = adata2.obs['cell_id'].map(mapping_dict_celltype)
adata2.obs['region_automap'] = adata2.obs['cell_id'].map(mapping_dict_region)
# adata2.obs['region_manual'] = adata.obs['region_manual_name']

all_celltype = np.array(adata2.obs['cell_type_final'].unique())

In [ ]:
# HC only
adata2.obs['Genotype'] = 0
adata2.obs['ZT'] = 0

geno_dict = {'3159-1':'WT','2670-1':'WT','3159-2':'WT','3159-3':'WT','3159-4':'WT','2505-1':'APP','2505-2':'APP','3160-1':'APP',
             '3160-2':'APP','3161-1':'APP','3161-2':'APP','3161-3':'APP',
}
ZT_dict = {
    '3159-1':'ZT17','2670-1':'ZT5','3159-2':'ZT17','3159-3':'ZT17','3159-4':'ZT17','2505-1':'ZT5','2505-2':'ZT5',
'3160-1':'ZT17','3160-2':'ZT17','3161-1':'ZT17','3161-2':'ZT17','3161-3':'ZT17',
}

adata2.obs['Genotype'] = adata2.obs['sample'].map(geno_dict)
adata2.obs['ZT'] = adata2.obs['sample'].map(ZT_dict)

In [ ]:
all_celltype = np.array(adata2.obs['cell_type_final'].unique())

In [ ]:
all_samples = adata2.obs['sample'].unique()

In [ ]:
all_copy = all_celltype
for item_ in all_celltype[:]:
    adata_microglia = adata2[adata2.obs['cell_type_final'] == item_]
    
    a = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[0]]) > 2
    b = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[1]]) > 2
    c = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[2]]) > 2
    d = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[3]]) > 2
    e = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[4]]) > 2
    f = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[5]]) > 2
    g = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[6]]) > 2
    h = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[7]]) > 2
    i = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[8]]) > 2
    j = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[9]]) > 2
    k = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[10]]) > 2
    l = len(adata_microglia.obs[adata_microglia.obs['sample'] == all_samples[11]]) > 2

    if not(a & b & c & d & e & f & g & h & i & j & k ):
        all_copy = np.delete(all_copy, np.where(all_copy == item_))        

all_celltype = all_copy

In [ ]:
sc.pp.normalize_total(adata2, inplace=True)
sc.pp.log1p(adata2)

In [ ]:
region_to_ext = 'MH Glut'

adata_subset_region = adata_subset[adata_subset.obs['region']==region_to_ext]

all_celltype = np.array(adata_subset_region.obs['cell_type_final'].unique())

### Remove celltype not present in all group ---- to run until stops returning error
for i in range(0, len(all_celltype)):
    adata_microglia = adata2[adata2.obs['cell_type_final'] == all_celltype[i]]
    if ((len(adata_microglia.obs[adata_microglia.obs['Genotype'] == 'WT']) > 1) & (len(adata_microglia.obs[adata_microglia.obs['Genotype'] == 'APP']) > 1)):
        print('a')
    else:
        all_celltype = np.delete(all_celltype, [i])

In [ ]:
all_celltype = np.delete(all_celltype, np.where(all_celltype == 'AD Glut'))
all_celltype

In [ ]:
adata2.obs[adata2.obs['cell_type_final'] == "AD Glut"]['sample'].unique()


In [ ]:
dfs = []

a_ = 0
strerhyt = '-'
strenbis = ' '

for cell_type_to_extract in all_celltype:
    adata_microglia = adata2[adata2.obs['cell_type_final'] == cell_type_to_extract]
    
    ### Extract gene expression per cluster + log fold change + p-value
    cluster_to_use = 'Genotype'
    clust_uniq = adata2.obs[cluster_to_use].unique()
    dat = pd.DataFrame()

    if (len(adata_microglia[adata_microglia.obs[cluster_to_use] == clust_uniq[0]]) < 2) or (len(adata_microglia[adata_microglia.obs[cluster_to_use] == clust_uniq[1]]) < 2):
        dat['nothing'] = ['to see here']
        dfs.append(dat)
        a_ +=1
        print(f'[{a_ * strerhyt}{(len(all_celltype) - a_)*strenbis}] Cluster {a_} / {len(all_celltype)}')
        continue

    adata_microglia.obs[cluster_to_use] = adata_microglia.obs[cluster_to_use].astype(str)
    #sc.tl.dendrogram(adata, groupby = 'L04_newnum_subclassname')
    sc.tl.rank_genes_groups(adata_microglia, groupby=cluster_to_use, method="wilcoxon", tie_correct = True, pts = True)
    
    # sc.pl.rank_genes_groups_dotplot(adata_microglia, groupby=cluster_to_use, standard_scale="var", n_genes=5)
    
    for i in adata2.obs[cluster_to_use].unique():
        # print(f"Cluster {cell_type_to_extract}_{i}")
        dat1 = sc.get.rank_genes_groups_df(adata_microglia, group=i)
        dat1['group'] = i
        dat = pd.concat([dat, dat1])

    dat = dat[ ### Choose filters here
    (dat['pct_nz_group'] > 0.15) & #Percentage of cell expressing the gene
    (dat['pvals_adj']<= 0.05) & # adjusted p-value
    (abs(dat['logfoldchanges']) > 0.26) # logfoldchange
    ]
    a_ +=1
    print(f'[{a_ * strerhyt}{(len(all_celltype) - a_)*strenbis}] Cluster {a_} / {len(all_celltype)}')

    dfs.append(dat)
else:
    print('Extraction done')


### Extract normalized expression and clusters for individual cells
# if not os.path.exists(f"{dir_notebook}/csv/{name_dir}/foldchange"):
#    os.makedirs(f"{dir_notebook}/csv/{name_dir}/foldchange")
# dat.to_csv(f"{dir_notebook}/csv/{name_dir}/foldchange/{name_dir}_clusters_foldchange_{cell_type_to_extract}.csv")

In [ ]:
import xlsxwriter
writer = pd.ExcelWriter(f'csv/{name_dir}/all_celltypes_df_comb_bis.xlsx', engine='xlsxwriter')
for j in range(0,len(dfs)):
    dfs[j].to_excel(writer, sheet_name=all_celltype[j], index=False)

writer.close()

## Normalized gene counts with cell type and automap regions

In [ ]:
# adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz")
adata_norm = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_norm.h5ad.gz")


In [ ]:
len(adata_norm), len(adata)

In [ ]:
adata.obs.sample(2)

In [ ]:
df

In [ ]:
# adata_norm = adata
# df = pd.DataFrame(data=adata_norm.X.toarray(), index=adata_norm.obs_names, columns=adata_norm.var_names)
df['cell_id'] = df.index

# # # Create a dictionary to map old values to new values
# mapping_dict_region = dict(zip(adata.obs['cell_id'], adata.obs['region_automap_name']))
# # mapping_dict_region_manos = dict(zip(adata.obs['cell_id'], adata.obs['region_manual_name']))
# mapping_dict_celltype = dict(zip(adata.obs['cell_id'], adata.obs['cell_type_final']))
mapping_dict_manos = dict(zip(adata.obs['cell_id'], adata.obs['sample']))
# mapping_dict_run = dict(zip(adata.obs['cell_id'], adata.obs['run']))
# mapping_dict_geno = dict(zip(adata.obs['cell_id'], adata.obs['Genotype']))


# # # Use .map() function to rename cell contents in 'col1' based on mapping dictionary
# df['region_automap'] = df['cell_id'].map(mapping_dict_region)
# # df['region_manual_map'] = df['cell_id'].map(mapping_dict_region_manos)
# df['cell_type_final'] = df['cell_id'].map(mapping_dict_celltype)
# df['run'] = df['cell_id'].map(mapping_dict_run)
df['sample'] = df['cell_id'].map(mapping_dict_manos)
# # df['Genotype'] = df['cell_id'].map(mapping_dict_geno)
# df.dropna(subset=['cell_type_final'], inplace=True)

In [ ]:
mapping_dict_ZT = dict(zip(adata.obs['cell_id'], adata.obs['ZT']))
df['ZT'] = df['cell_id'].map(mapping_dict_ZT)

In [ ]:
df.sample(2)

In [ ]:
### Save all normalized count + metadata as a single file

# df.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_norm_combined.csv.gz",
#             compression={'method': 'gzip'}
#          )

df.to_parquet(f"{dir_notebook}/csv/{name_dir}/{name_dir}_norm_combined_regions.parquet")

In [ ]:
### Extract normalized gene count for each samples separately (for a subset of markers genes, uncomment filter)
from module.misc import 

markers_genes = ['cell_id','Arntl',"Ciart","Clock","Cry1","Cry2","Dbp","Hlf",
                "Nfil3","Npas2","Nr1d1","Per1","Per2","Per3","Rora",
                "Rorb","Rorc"]



count = 0
for sample_idd in samples_ids:
    count += 1
    print(f'{count}/{len(samples_ids)}')
    df_temp = df[df['sample'] == sample_idd]
    df_temp = df_temp.filter(markers_genes, axis=1)
    # df['region_automap'] = df['cell_id'].map(mapping_dict_region)
    df_temp['cell_type_final'] = df_temp['cell_id'].map(mapping_dict_celltype)
    df_temp['sample'] = df_temp['cell_id'].map(mapping_dict_manos)
    
    df_temp.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_norm_combined_{sample_idd}_allgenes.csv.gz",
            compression={'method': 'gzip'})



In [ ]:
### Extract percentage of cells expressing a set of genes in all celltypes

markers_genes = ['Arntl',"Ciart","Clock","Cry1","Cry2","Dbp","Hlf",
                "Nfil3","Npas2","Nr1d1","Per1","Per2","Per3","Rora",
                "Rorb","Rorc"]

all_celltype = df['cell_type_final'].unique()
all_celltype = np.append(all_celltype, 'All')

count = 0
prct_all = pd.DataFrame([], index=markers_genes)

for celltype_ in all_celltype:
    count += 1
    print(f'{count}/{len(all_celltype)}')

    if celltype_ != 'All':
        df_temp = df[df['cell_type_final'] == celltype_]
        df_temp2 = df_temp.filter(markers_genes, axis=1)
    else:
        df_temp2 = df.filter(markers_genes, axis=1)

    temp_pct = []
    for gene_ in markers_genes:
        abt = len(df_temp2[df_temp2[gene_] > 0]) / len(df_temp2)
        temp_pct.append(abt)
    
    prct_all[celltype_]= pd.DataFrame(temp_pct, index=markers_genes)

prct_all.to_csv(f'csv/{name_dir}/{name_dir}_CG_percentage.csv')

In [ ]:
all_region

In [ ]:
### Extract percentage of cells expressing a set of genes in all regions

markers_genes = ['Arntl',"Ciart","Clock","Cry1","Cry2","Dbp","Hlf",
                "Nfil3","Npas2","Nr1d1","Per1","Per2","Per3","Rora",
                "Rorb","Rorc"]

all_region = df['region_automap'].unique()
all_region = np.append(all_region, 'All')

count = 0
prct_all = pd.DataFrame([], index=markers_genes)

for celltype_ in all_region:
    count += 1
    print(f'{count}/{len(all_region)}')

    if celltype_ != 'All':
        df_temp = df[df['region_automap'] == celltype_]
        df_temp2 = df_temp.filter(markers_genes, axis=1)
    else:
        df_temp2 = df.filter(markers_genes, axis=1)

    temp_pct = []
    for gene_ in markers_genes:
        if len(df_temp2) !=0:
            abt = len(df_temp2[df_temp2[gene_] > 0]) / len(df_temp2)
        else:
            abt = 0
        temp_pct.append(abt)
    
    prct_all[celltype_]= pd.DataFrame(temp_pct, index=markers_genes)

prct_all.to_csv(f'csv/{name_dir}/{name_dir}_CG_regions_percentage.csv')

In [ ]:
celltype_ = adata.obs['cell_type_final'].unique()
# celltype_ = adata.obs['cell_type_final'].unique()[0:1]
gene_dfs = []
# df2 = df
# marker_genes = adata_norm.var.index
marker_genes = ['Arntl',"Ciart","Clock","Cry1","Cry2","Dbp","Hlf",
                "Nfil3","Npas2","Nr1d1","Per1","Per2","Per3","Rora",
                "Rorb","Rorc"]
genes_df = pd.DataFrame()
sample_list = pd.DataFrame(adata.obs['ZT'].unique())

for celltype in celltype_:
    df2 = df[df['cell_type_final'] == celltype]
    genes_df = pd.DataFrame()
    for gene_ in marker_genes:
        temp = df2.groupby('ZT')[gene_].mean()
        temp_df = pd.DataFrame(temp)
        genes_df = pd.concat([genes_df, temp_df], axis = 1)
    gene_dfs.append(genes_df)


import xlsxwriter
writer = pd.ExcelWriter(f'{dir_notebook}/csv/{name_dir}/{name_dir}_gene_dfs.xlsx', engine='xlsxwriter')
for j in range(0,len(gene_dfs)):
    gene_dfs[j].to_excel(writer, sheet_name=celltype_[j], index=False)

writer.close()

In [ ]:
import xlsxwriter

runs_ = df['run'].unique()
# celltype_ = adata.obs['cell_type_final'].unique()
# celltype_ = adata.obs['cell_type_final'].unique()[0:1]
gene_dfs = []
# df2 = df
# marker_genes = adata_norm.var.index
marker_genes = ['Arntl',"Ciart","Clock","Cry1","Cry2","Dbp","Hlf",
                "Nfil3","Npas2","Nr1d1","Per1","Per2","Per3","Rora",
                "Rorb","Rorc"]
genes_df = pd.DataFrame()
sample_list = pd.DataFrame(adata.obs['ZT'].unique())

for run in runs_:
    genes_df = pd.DataFrame()
    df3 = df[df['run'] == run]
    celltype_ = df3['cell_type_final'].unique()
    for celltype in celltype_:
        df2 = df3[df3['cell_type_final'] == celltype]
        genes_df = pd.DataFrame()
        for gene_ in marker_genes:
            temp = df2.groupby('ZT')[gene_].mean()
            temp_df = pd.DataFrame(temp)
            genes_df = pd.concat([genes_df, temp_df], axis = 1)
        gene_dfs.append(genes_df)


    writer = pd.ExcelWriter(f'data/{name_dir}_gene_dfs_{run}.xlsx', engine='xlsxwriter')
    for j in range(0, len(celltype_)):
        # print(j)
        gene_dfs[j].to_excel(writer, sheet_name=celltype_[j], index=True)

    writer.close()

In [ ]:
plt.plot(gene_dfs[9]['Per2'])

In [ ]:
import numpy as np

from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt 

x = np.array([1, 5, 9, 13, 17, 21])
y = np.array(gene_dfs[9]['Per2'])

X_Y_Spline = make_interp_spline(x, y)

# Returns evenly spaced numbers
# over a specified interval.
X_ = np.linspace(x.min(), x.max(), 50)
Y_ = X_Y_Spline(X_)


plt.plot(X_, Y_)
plt.xlabel("ZT")
plt.ylabel('Average expression (log2)')
# plt.ylim((0,Y_.max()+))
plt.title("Per2 expression in SCN neurons")
plt.show()

In [ ]:
df.groupby('ZT')['Per1'].mean()

In [ ]:
gene_ = 'Gfap'

y1 = df[df['Genotype'] == 'WT'][gene_].sort_values()#.cumsum()
y3 = y1[y1>0]
x1 = range(0,len(y1))
x3 = range(0,len(y3))
y2 = df[df['Genotype'] == 'APP'][gene_].sort_values()#.cumsum()
y4 = y2[y2>0]
x2 = range(0,len(y2))
x4 = range(0,len(y4))
plt.scatter(x1, y1,c='blue')
plt.scatter(x2,y2,c='red')
plt.scatter(x4,y4,c='red')
#plt.title(gene_)
plt.ylabel('')
plt.xlabel('# cells')
# plt.xscale()
# plt.yscale("log")

plt.show()

In [ ]:

# sns.histplot(y1, stat = 'percent', label = 'WT', kde=True, color = 'blue')
sns.histplot(y3, stat = 'percent',  kde=True, color = 'darkblue')
sns.histplot(y4, stat = 'percent',  kde=True, color = 'darkred')
# sns.histplot(y2, stat = 'percent', label = 'APP', kde=True, color = 'red')
plt.xlim(0.05,)
plt.ylim(0,5)
plt.title = gene_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Assume x and y are the data points
x = x3
y = y3

# Fit a smooth polynomial (e.g., degree 5)
coeffs = np.polyfit(x, y, deg=3)
smooth_function = np.poly1d(coeffs)

# Compute derivatives
first_derivative = np.polyder(smooth_function)
second_derivative = np.polyder(first_derivative)

# Find stationary points (where first derivative is zero)
x_vals = np.linspace(min(x), max(x), len(x3))
first_derivative_values = first_derivative(x_vals)

# x_vals_sec = np.linspace(min(x), max(x), len(x3))
# sec_derivative_values = second_derivative(x_vals_sec)


# # Identify points where first derivative is approximately zero
# stationary_points = x_vals_sec[np.isclose(sec_derivative_values, 0, atol=1e-5)]

# Identify points where first derivative is approximately zero
stationary_points = x_vals_sec[np.isclose(first_derivative_values, 0, atol=1e-5)]

# Classify stationary points using the second derivative
stationary_types = []
for point in stationary_points:
    second_derivative_value = second_derivative(point)
    if second_derivative_value > 0:
        stationary_types.append("Minimum")
    elif second_derivative_value < 0:
        stationary_types.append("Maximum")
    else:
        stationary_types.append("Saddle Point")

# Plot the results
plt.plot(x, y, label="Original Curve")
plt.scatter(stationary_points, smooth_function(stationary_points), color='red', label="Stationary Points")
plt.legend()
plt.show()

# Print the stationary points and their classification
# for i, point in enumerate(stationary_points):
#     print(f"Stationary Point at x = {point:.2f}: {stationary_types[i]}")

In [ ]:
# plt.scatter(x = range(len(first_derivative_values)), y=first_derivative_values)
plt.scatter(x = range(len(sec_derivative_values)), y=sec_derivative_values)

In [ ]:
plt.scatter(x=x3, y=y3)

In [ ]:
xdata=x3
ydata=y3

def func(x, a, b, c):

    return a * np.exp(b * x) + c

popt, pcov = curve_fit(func, xdata, ydata)

plt.scatter(x=x3,y=y3)
plt.plot(xdata, func(xdata, *popt), 'r-',)

## Extract score, FC, p-value for each genes, compared to other clusters

In [ ]:
### Extract gene expression per cluster + log fold change + p-value

cluster_to_use = 'sample'

adata2.obs[cluster_to_use] = adata2.obs[cluster_to_use].astype(str)
#sc.tl.dendrogram(adata, groupby = 'L04_newnum_subclassname')
sc.tl.rank_genes_groups(adata2, groupby=cluster_to_use, method="wilcoxon", tie_correct = True)

# sc.pl.rank_genes_groups_dotplot(adata2, groupby=cluster_to_use, standard_scale="var", n_genes=5)

dat = pd.DataFrame()
for i in adata2.obs[cluster_to_use].unique():
    print(f"Cluster {i}")
    dat1 = sc.get.rank_genes_groups_df(adata2, group=str(i))
    dat1['group'] = i
    dat = pd.concat([dat, dat1])

### Extract not-normalized expression and clusters for individual cells
if not os.path.exists(f"{dir_notebook}/csv/{name_dir}/"):
   os.makedirs(f"{dir_notebook}/csv/{name_dir}/")
dat.to_csv(f"{dir_notebook}/csv/{name_dir}/{name_dir}_clusters_foldchange_allbrain.csv")

# Test zone

## Outliers and highly variable genes detection

### Outliers

In [ ]:
# from scipy.stats import median_abs_deviation

# def is_outlier(adata, metric: str, nmads: int):
#     M = adata.obs[metric]
#     outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
#         M > np.median(M) + nmads * median_abs_deviation(M)
#     )
#     return outlier

In [ ]:
# np.median(adata.obs["log1p_total_counts"]) - 5 * median_abs_deviation(adata.obs["log1p_total_counts"])

In [ ]:
# np.median(adata.obs["log1p_n_genes_by_counts"]) - 5 * median_abs_deviation(adata.obs["log1p_n_genes_by_counts"])

In [ ]:
# adata.obs["outlier"] = (
#     is_outlier(adata, "log1p_total_counts", 5)
#     | is_outlier(adata, "log1p_n_genes_by_counts", 5)
#     | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
# )

In [ ]:
# adata.obs[adata.obs['outlier'] == True][['log1p_total_counts',"log1p_n_genes_by_counts","pct_counts_in_top_20_genes",'outlier']].sample(12)

In [ ]:
# pd.set_option("display.max_row", None)
# adata.obs[adata.obs['outlier'] == True].groupby('mmc:subclass_name')['cell_id'].nunique()

In [ ]:
# adata.obs[adata.obs['outlier'] == True].shape

In [ ]:
# adata.obs[adata.obs['mmc:subclass_name'] == "334 Microglia NN"]['cell_id'].nunique()

### Highly variable genes

In [ ]:
# sc.pp.highly_variable_genes(adata)

In [ ]:
# adata.var[adata.var['highly_variable']==True].

## Match cells with manually drawn regions (cell id)

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz")

In [ ]:
hipp_df = pd.read_csv(f'{dir_notebook}/coordinates/Hipp_run3.csv')

In [ ]:
hipp_df.head(2)

In [ ]:
# Create a dictionary to map old values to new values
mapping_dict = dict(zip(hipp_df['cell_id'], hipp_df['region']))

# Use .map() function to rename cell contents in 'col1' based on mapping dictionary
adata.obs['region_manos'] = adata.obs['cell_id'].apply(lambda x: mapping_dict[x] if x in mapping_dict else np.nan)

In [ ]:
adata.obs.loc[
    (adata.obs['region_manos'] == 'Hipp_manos') &
    (adata.obs['region_automap_name'] == 'HIPP'), 'test'] = "_granular"
adata.obs.loc[
    (adata.obs['region_manos'] == 'Hipp_manos') &
    (adata.obs['region_automap_name'].isnull()), 'test'] = "_molecular"
adata.obs.loc[
    (adata.obs['region_manos'].isnull()) &
    (adata.obs['region_automap_name'] == 'HIPP'), 'test'] = "_granular"

In [ ]:
adata.obs.sample(50)

In [ ]:
# Create a dictionary to map old values to new values
mapping_dict = dict(zip(adata.obs[adata.obs['test'] != "0"]['cell_id'], adata.obs[adata.obs['test'] != "0"]['test']))

# Use .map() function to rename cell contents in 'col1' based on mapping dictionary
adata.obs[adata.obs['test'].notnull()]['region_automap_name'] = adata.obs['cell_id'].apply(lambda x: mapping_dict[x] if x in mapping_dict else x)

In [ ]:
all_region_type = adata.obs['region_automap_name'].unique()
list_region_nb = range(0, len(all_region_type))
mapping_dict = dict(zip(all_region_type,list_region_nb))
adata.obs['region_newnum'] = adata.obs['region_automap_name'].map(mapping_dict)
# mapping_dict

In [ ]:
adata.write(f"{dir_notebook}/h5ad/{name_dir}/{name_dir}_MMC_Banksy_annotated_automap.h5ad.gz", compression = 'gzip')

## Plaques

See Untitled.ipynd (temporary name, hopefully)

In [ ]:
# adata = sc.read_h5ad(f"{dir_notebook}/h5ad/run1-resegment/run1-resegment_MMC_Banksy_annotated_automap.h5ad.gz")
# adata2 = sc.read_h5ad(f"{dir_notebook}/h5ad/run3-Habenula/run3-Habenula_MMC_Banksy_annotated_automap.h5ad.gz")
# adata3 = sc.read_h5ad(f"{dir_notebook}/h5ad/run3-LGN/run3-LGN_MMC_Banksy_annotated_automap.h5ad.gz")
# adata4 = sc.read_h5ad(f"{dir_notebook}/h5ad/run3-SC/run3-SC_MMC_Banksy_annotated_automap.h5ad.gz")

In [ ]:
adata

In [ ]:
adata.obs.rename({"n_couts":'total_counts',})

In [ ]:
adata.obs = adata.obs.filter(['cell_id','total_counts','n_genes','cell_area', ], axis=1)
adata2.obs = adata2.obs.filter(['cell_id','total_counts','n_genes','cell_area'], axis=1)
adata3.obs = adata3.obs.filter(['cell_id','total_counts','n_genes','cell_area'], axis=1)
adata4.obs = adata4.obs.filter(['cell_id','total_counts','n_genes','cell_area'], axis=1)

In [ ]:
adatas = adata.concatenate(adata2, adata3, adata4)

In [ ]:
adatas

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 3))
plt.subplots_adjust(wspace=0.3)

axs[0].set_title("Total transcripts per cell")
sns.histplot(adata.obs["total_counts"],kde=False,ax=axs[0], stat = "count")

axs[1].set_title("Unique transcripts per cell")
sns.histplot(adata.obs["n_genes"],kde=False,ax=axs[1])

axs[2].set_title("Area of segmented cells")
sns.histplot(adata.obs["cell_area"], kde=False, ax=axs[2])

# axs[3].set_title("Nucleus ratio")
# sns.histplot(adata.obs["nucleus_area"] / adata.obs["cell_area"], kde=False,ax=axs[3])

# if not os.path.exists(f"{dir_notebook}/plot/{name_dir}/"):
#    os.makedirs(f"{dir_notebook}/plot/{name_dir}/")
plt.savefig(f"alldata_quality-metrics.svg")

In [ ]:
adatas.obs['cell_area'].median(),adatas.obs['cell_area'].mean(),adatas.obs['cell_area'].min(),adatas.obs['cell_area'].max()

In [ ]:
adatas.obs['total_counts'].sum()

In [ ]:
adata.var

In [ ]:
def celltype_heatmap(adata, colors, figsize=(8, 8)):
    # Rank gene group based on celltype
    sc.tl.rank_genes_groups(
        adata,
        groupby="cell type",
        use_raw=False,
        method="wilcoxon",
        corr_method="bonferroni",
    )
    
    # Create dendrorgam
    sc.tl.dendrogram(
        adata,
        groupby="cell type",
        use_raw=False,
        cor_method="pearson"
    )
    
    # Run sc.pl.rank_genes_groups_heatmap once to create adata.uns["celltype_colors"] object
    sc.pl.rank_genes_groups_heatmap(
        adata,
        show_gene_labels=False,
        use_raw=False,
        show=False
    )
    plt.close()

    # Relabel celltype colors
    adata.uns["celltype_colors"] = colors
    
    # Plot heatmap with dendrogram
    sc.pl.rank_genes_groups_heatmap(
        adata,
        show_gene_labels=True,
        use_raw=False,
        cmap="inferno",
        standard_scale="var",
        figsize=figsize
    )

In [ ]:
celltype_heatmap(adata,adata.obs['leiden_colors'])

In [ ]:
palette

In [ ]:
adata.obs[adata.obs['cell type']=='SO Glut']['mmc:subclass_name']

## Concatenate MMC and h5ad

In [ ]:
csv1 = pd.read_csv('Correlation_Mapping/nuclei_3159-2_CorrelationMapping.csv', comment = '#')
csv2 = pd.read_csv('Correlation_Mapping/nuclei_3161-1_CorrelationMapping.csv', comment =  '#')
# csv3 = pd.read_csv('Correlation_Mapping/run3-LGN_CorrelationMapping.csv', comment =  '#')
# csv4 = pd.read_csv('Correlation_Mapping/run3-SC_CorrelationMapping.csv', comment =  '#')
corr = pd.concat([csv1, csv2,
                  #csv3, csv4
                 ], ignore_index=True)

In [ ]:
adata = sc.read_h5ad(f"{dir_notebook}/h5ad/run3/nuclei_3161-1_forMMC.h5ad")
adata.obs = adata.obs.loc[:, adata.obs.columns.intersection(['cell_id','transcript_counts','x_centroid','y_centroid','sample'])]
adata.obs_names = [f"3161-1_{cell_id}" for cell_id in adata.obs_names]
adata.obs['cell_id'] = adata.obs_names

adata2 = sc.read_h5ad(f"{dir_notebook}/h5ad/run3/nuclei_3159-2_forMMC.h5ad")
adata2.obs = adata2.obs.loc[:, adata2.obs.columns.intersection(['cell_id','transcript_counts','x_centroid','y_centroid','sample'])]
adata2.obs_names = [f"3159-2_{cell_id}" for cell_id in adata2.obs_names]
adata2.obs['cell_id'] = adata2.obs_names

# adata3 = sc.read_h5ad(f"{dir_notebook}/h5ad/run3-LGN/run3-LGN_forMMC.h5ad")
# adata3.obs = adata3.obs.loc[:, adata3.obs.columns.intersection(['cell_id','transcript_counts','x_centroid','y_centroid','sample'])]

# adata4 = sc.read_h5ad(f"{dir_notebook}/h5ad/run1-resegment/run1-resegment_forMMC.h5ad")
# adata4.obs['transcript_counts'] = adata4.obs['total_transcript']
# adata4.obs = adata4.obs.loc[:, adata4.obs.columns.intersection(['cell_id','transcript_counts','x_centroid','y_centroid','sample'])]

adata_all = adata.concatenate([adata2,
                              # adata3,adata4
                              ], index_unique=None)
adata_all.var = adata_all.var.loc[:, adata_all.var.columns.intersection(['gene_ids','feature_types','genome'])]
adata_all.obs.index.name = None

In [ ]:
# HC3_MMC = corr
HC3_MMC = pd.read_csv('Correlation_Mapping/nuclei_run3_CorrelationMapping.csv', comment = '#')
HC3_MMC.index = HC3_MMC['cell_id']
HC3_MMC.index.name = None
HC3_MMC.columns = [f"mmc:{i}" for i in HC3_MMC.columns]
mmc_dict_class = dict(zip(HC3_MMC['mmc:cell_id'], HC3_MMC['mmc:class_name']))
mmc_dict_classcoef = dict(zip(HC3_MMC['mmc:cell_id'], HC3_MMC['mmc:class_correlation_coefficient']))
mmc_dict_subclass = dict(zip(HC3_MMC['mmc:cell_id'], HC3_MMC['mmc:subclass_name']))
mmc_dict_supertype = dict(zip(HC3_MMC['mmc:cell_id'], HC3_MMC['mmc:supertype_name']))

adata_all.obs['mmc:class_name'] = adata_all.obs['cell_id'].map(mmc_dict_class)
adata_all.obs['mmc:class_correlation_coefficient'] = adata_all.obs['cell_id'].map(mmc_dict_classcoef)
adata_all.obs['mmc:subclass_name'] = adata_all.obs['cell_id'].map(mmc_dict_subclass)
adata_all.obs['mmc:supertype_name'] = adata_all.obs['cell_id'].map(mmc_dict_supertype)

In [ ]:
adata2.obs

In [ ]:
adata_all.write_h5ad(f'h5ad/run1/nuclei.h5ad.gz', compression = 'gzip')

In [ ]:
adata = adata_all

In [ ]:
adata2.obs['cell_id'] = adata2.obs_names

In [ ]:
adata_all.write_h5ad(f"{dir_notebook}/h5ad/run3/nuclei_run3_forMMC.h5ad")

In [ ]:
plt.scatter(adata2.obs['x_centroid'],adata2.obs['y_centroid'], s= 0.05)

In [ ]:
HC3_MMC

# Test graphs

In [ ]:
adata.uns

In [ ]:
import squidpy as sq
adata_s1 = adata[adata.obs['sample']=='3159-2']
sq.pl.spatial_scatter(adata_s1,
                     spatial_key = "coord_xy",
                     color = ['Gfap'],
                     shape = "circle",
                     size=2,
                     img = False)

In [ ]:
import spatialdata as sd

In [ ]:
adata_s1.pl.render_shapes("cell_circles",color ='Gfap',)

In [ ]:
adata.obs['nuc_mmc:subclass_name'].unique()

In [ ]:
### Correlation map
cont_tab = pd.crosstab(adata.obs['nuc_mmc:class_name'], adata.obs['mmc:class_name'], normalize="index")
plt.figure(figsize=(300, 15))
sns.heatmap(cont_tab, annot=True, cmap="YlGnBu", fmt=".1f")

In [ ]:
# Generate new numbering base on unique 'cell type'
all_cell_type = adata.obs['nuc_mmc:class_name'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['cell_type_newnum'] = adata.obs['nuc_mmc:class_name'].map(mapping_dict)
mapping_dict

In [ ]:
### Check clusters one by one to see if they are present in all sample and which would need subclustering

cluster_to_use = 'cell_type_newnum'

### Generate a color palette for the clusters - to make color stay consistent across samples
adata.obs[cluster_to_use] = adata.obs[cluster_to_use].astype(str)

# Create a palette with a unique color for each cluster
num_clusters = len(adata.obs[cluster_to_use].astype(int).unique())
palette = sns.color_palette("tab20b", n_colors=num_clusters +1)

# Map each 'leiden' value to a color
adata.obs['leiden_colors'] = adata.obs[cluster_to_use].astype(int).apply(lambda x: palette[x])

# Map all cells
fig, axs = plt.subplots(3,2,figsize=(14, 12))
axs = axs.flatten()
clusters_plot = {"19":'black', ### For VLMC
    # '0': 'lightcoral',"1": 'forestgreen', '2':'red', "3":'purple', "4":"yellow"
    # '5': 'lightcoral',"6": 'forestgreen', 7:'red', "8":'purple', "9":"yellow"
    # '10': 'lightcoral',"11": 'forestgreen', '12':'red', "13":'purple', "15":"yellow", "14": "blue"
    # '16': 'lightcoral',"17": 'forestgreen', '18':'red', "19":'purple', "20":"yellow"
      # '21': 'lightcoral',"22": 'forestgreen', '23':'red', "":'purple', "":"yellow",
    # '24': 'lightcoral',"25": 'forestgreen', '26':'red', "27":'purple', "28":"yellow"
    # '29': 'lightcoral',"30": 'forestgreen', '31':'red', "32":'purple', "33":"yellow"
    # '34': 'lightcoral',"35": 'forestgreen', '36':'red', "37":'purple', "38":"yellow", "39": "blue"
    # '2': 'lightcoral',"6": 'forestgreen', '13':'red', "19":'purple', "":"yellow"
    # '35': 'lightcoral',"20": 'forestgreen', '':'red', "":'purple', "":"yellow"
                }

for idx, sample in enumerate(samples_ids):
    adata_sel = adata[(adata.obs['sample'] == sample)]
    for cluster_id in adata_sel.obs[cluster_to_use].unique():
        cluster_data = adata_sel.obs[adata_sel.obs[cluster_to_use] == cluster_id]
        colors = clusters_plot[cluster_id] if cluster_id in clusters_plot else "none" ### for selected clusters in cluster_plot
        # colors= cluster_data['leiden_colors'].unique()[0] ### uncomment for all clusters
        axs[idx].scatter(cluster_data['x_centroid'], cluster_data['y_centroid'], color=colors, s=0.25, label=cluster_id)
        axs[idx].set_title(f"Sample {sample}")

In [ ]:
# Generate new numbering base on unique 'cell type'
all_cell_type = adata.obs['mmc:class_name'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['cell_type_newnum2'] = adata.obs['mmc:class_name'].map(mapping_dict)
mapping_dict

In [ ]:
### Check clusters one by one to see if they are present in all sample and which would need subclustering

cluster_to_use = 'cell_type_newnum2'

### Generate a color palette for the clusters - to make color stay consistent across samples
adata.obs[cluster_to_use] = adata.obs[cluster_to_use].astype(str)

# Create a palette with a unique color for each cluster
num_clusters = len(adata.obs[cluster_to_use].astype(int).unique())
palette = sns.color_palette("tab20b", n_colors=num_clusters +1)

# Map each 'leiden' value to a color
adata.obs['leiden_colors'] = adata.obs[cluster_to_use].astype(int).apply(lambda x: palette[x])

# Map all cells
fig, axs = plt.subplots(3,2,figsize=(14, 12))
axs = axs.flatten()
clusters_plot = {"18":'black', ### For VLMC
    # '0': 'lightcoral',"1": 'forestgreen', #'2':'red', "3":'purple', "4":"yellow"
    # '5': 'lightcoral',"6": 'forestgreen', 7:'red', "8":'purple', "9":"yellow"
    # '10': 'lightcoral',"11": 'forestgreen', '12':'red', "13":'purple', "15":"yellow", "14": "blue"
    # '16': 'lightcoral',"17": 'forestgreen', '18':'red', "19":'purple', "20":"yellow"
      # '21': 'lightcoral',"22": 'forestgreen', '23':'red', "":'purple', "":"yellow",
    # '24': 'lightcoral',"25": 'forestgreen', '26':'red', "27":'purple', "28":"yellow"
    # '29': 'lightcoral',"30": 'forestgreen', '31':'red', "32":'purple', "33":"yellow"
    # '34': 'lightcoral',"35": 'forestgreen', '36':'red', "37":'purple', "38":"yellow", "39": "blue"
    # '2': 'lightcoral',"6": 'forestgreen', '13':'red', "19":'purple', "":"yellow"
    # '35': 'lightcoral',"20": 'forestgreen', '':'red', "":'purple', "":"yellow"
                }

for idx, sample in enumerate(samples_ids):
    adata_sel = adata[(adata.obs['sample'] == sample)]
    for cluster_id in adata_sel.obs[cluster_to_use].unique():
        cluster_data = adata_sel.obs[adata_sel.obs[cluster_to_use] == cluster_id]
        colors = clusters_plot[cluster_id] if cluster_id in clusters_plot else "none" ### for selected clusters in cluster_plot
        # colors= cluster_data['leiden_colors'].unique()[0] ### uncomment for all clusters
        axs[idx].scatter(cluster_data['x_centroid'], cluster_data['y_centroid'], color=colors, s=0.25, label=cluster_id)
        axs[idx].set_title(f"Sample {sample}")

In [ ]:
adata.obs.groupby('mmc:subclass_name')['mmc:subclass_name'].count().sort_values()

In [ ]:
adata.obs.groupby('nuc_mmc:subclass_name')['nuc_mmc:subclass_name'].count().sort_values()

In [ ]:
# Obtain cluster-specific differentially expressed genes
# cluster_to_use = 'cell_type_newnum'
cluster_to_use = 'nuc_mmc:class_name'
adata.obs[cluster_to_use] = adata.obs[cluster_to_use].astype(str)
sc.tl.rank_genes_groups(adata, groupby=cluster_to_use, method="wilcoxon", tie_correct = True)
sc.pl.rank_genes_groups_dotplot(adata, groupby=cluster_to_use, standard_scale="var", n_genes=2)

In [ ]:
adata2

In [ ]:
test_ = 'nucleus_count'
adata2.obs[test_].median(),adata2.obs[test_].mean(),adata2.obs[test_].min(),adata2.obs[test_].max()

In [ ]:
adata.obs[adata.obs['nucleus_count'] == 4]

# Archive def functions

In [ ]:
# df['ZT'] = df['sample'].map(lambda name: name.split('-')[-1])
plt.scatter(x=df['ZT'].unique(), y=df[df['cell_type_final'] == 'SCH Gaba'].groupby('ZT')['Dbp'].mean())
plt.tick_params(axis = 'x', rotation = 90)

In [ ]:
### Automatic subclustering
mapping_dict_all = dict(zip(adata.obs['cell_id'], adata.obs['cell_type_auto']))
clusters_numbers = 'cell_type_newnum_auto'
# clusters_numbers = 'cell_type_newnum_auto_sub'
all_types = adata.obs[clusters_numbers].unique() ### Subcluster all clusters
# all_types = ['20'] ### only subcluster the selected clusters (might need to be between comas (as string)) --- Comment to run all clusters

for cluster_to_sub in (all_types):
    print(f'Subclustering of cluster {cluster_to_sub} / {len(all_types)-1}')
    adata_subcluster = adata_filter[adata_filter.obs[clusters_numbers] == cluster_to_sub]
    sc.pp.pca(adata_subcluster)
    sc.pp.neighbors(adata_subcluster)
    sc.tl.umap(adata_subcluster)
    sc.tl.leiden(adata_subcluster, resolution = 0.1)

    clustering_method = 'leiden'
    n = len(adata_subcluster.obs[clustering_method].unique())
    print(f'Results: {n} clusters')

    # adata_subcluster.obs['new_cluster'] = 0
    adata_subcluster.obs['new_cluster2'] = adata_subcluster.obs[clustering_method].astype("str")
    adata_subcluster.obs['new_cluster_class2'] = adata_subcluster.obs[clustering_method].astype("str")

    cont_tab = pd.crosstab(adata_subcluster.obs[clustering_method], adata_subcluster.obs['mmc:subclass_name'], normalize="index")
    max_col_dict = cont_tab.T.idxmax(axis=0).to_dict()

    adata_subcluster.obs['cell_type_auto'] = adata_subcluster.obs['new_cluster2'].map(max_col_dict)

    # Create a dictionary to map old values to new values
    mapping_dict = dict(zip(adata_subcluster.obs['cell_id'], adata_subcluster.obs['cell_type_auto']))

    mapping_dict_all.update(mapping_dict)

# Use .map() function to rename cell contents in 'col1' based on mapping dictionary
adata.obs['cell_type_auto_sub'] = adata.obs['cell_id'].map(mapping_dict_all)

all_cell_type = adata.obs['cell_type_auto_sub'].unique()
list_cell_nb = range(0, len(all_cell_type))
mapping_dict = dict(zip(all_cell_type,list_cell_nb))
adata.obs['cell_type_newnum_auto_sub'] = adata.obs['cell_type_auto_sub'].map(mapping_dict)

In [ ]:
def auto_subclustering(adata_to_sub, all_types = 'all', Clusters_to_use = 'cell_type_newnum_auto', resolution = 0.1, ):
    ### Filters out rare class name
    adata_filter = adata_to_sub
    
    ### Automatic subclustering
    if Clusters_to_use == 'cell_type_newnum_auto':
        mapping_dict_all = dict(zip(adata.obs['cell_id'], adata.obs['cell_type_auto']))
    elif Clusters_to_use == 'cell_type_newnum_auto_sub':
        mapping_dict_all = dict(zip(adata.obs['cell_id'], adata.obs['cell_type_auto_sub']))
    
    clusters_numbers = Clusters_to_use
    
    if all_types == 'all':
        all_types = adata.obs[clusters_numbers].unique() ### Subcluster all clusters
    current_ = 0
    for cluster_to_sub in (all_types):
        total_cluster = str(len(all_types))
        current_ += 1
        print(f'Subclustering of cluster {cluster_to_sub} ({current_} / {total_cluster})')
        adata_subcluster = adata_filter[adata_filter.obs[clusters_numbers] == cluster_to_sub]
        sc.pp.pca(adata_subcluster)
        sc.pp.neighbors(adata_subcluster)
        sc.tl.umap(adata_subcluster)
        sc.tl.leiden(adata_subcluster, resolution = 0.1)

        clustering_method = 'leiden'
        n = len(adata_subcluster.obs[clustering_method].unique())
        print(f'Results: {n} clusters')

        # adata_subcluster.obs['new_cluster'] = 0
        adata_subcluster.obs['new_cluster2'] = adata_subcluster.obs[clustering_method].astype("str")
        adata_subcluster.obs['new_cluster_class2'] = adata_subcluster.obs[clustering_method].astype("str")

        cont_tab = pd.crosstab(adata_subcluster.obs[clustering_method], adata_subcluster.obs['mmc:subclass_name'], normalize="index")
        max_col_dict = cont_tab.T.idxmax(axis=0).to_dict()

        adata_subcluster.obs['cell_type_auto'] = adata_subcluster.obs['new_cluster2'].map(max_col_dict)

        # Create a dictionary to map old values to new values
        mapping_dict = dict(zip(adata_subcluster.obs['cell_id'], adata_subcluster.obs['cell_type_auto']))

        mapping_dict_all.update(mapping_dict)

    # Use .map() function to rename cell contents in 'col1' based on mapping dictionary
    adata.obs['cell_type_auto_sub'] = adata.obs['cell_id'].map(mapping_dict_all)

    all_cell_type = adata.obs['cell_type_auto_sub'].unique()
    list_cell_nb = range(0, len(all_cell_type))
    mapping_dict = dict(zip(all_cell_type,list_cell_nb))
    adata.obs['cell_type_newnum_auto_sub'] = adata.obs['cell_type_auto_sub'].map(mapping_dict)

In [ ]:

def cluster_table(adata_to_use = adata, Clusters_to_use = 'cell_type_newnum_auto_sub', sort_order = 'Cell Count', sort_ascend = False):

    Clusters_to_use = 'cell_type_newnum_auto_sub'
    # Clusters_to_use = 'cell_type_newnum_auto'
    # Clusters_to_use = 'cell_type_newnum_final'
    adata_to_use.obs[Clusters_to_use] = adata_to_use.obs[Clusters_to_use].astype(int)

    if Clusters_to_use == 'cell_type_newnum_auto_sub':
        Clusters_to_use_cells = 'cell_type_auto_sub'
    elif Clusters_to_use == 'cell_type_newnum_auto':
        Clusters_to_use_cells = 'cell_type_auto'
    elif Clusters_to_use == 'cell_type_newnum_final':
        Clusters_to_use_cells = 'cell_type_final'


    cont_tab_sub = pd.crosstab(adata_to_use.obs[Clusters_to_use],adata_to_use.obs['mmc:subclass_name'], normalize="index")
    cont_tab_sub = cont_tab_sub.loc[:, cont_tab_sub.sum(axis=0) > 0.05]

    cont_tab = pd.crosstab(adata_to_use.obs[Clusters_to_use], adata_to_use.obs['mmc:class_name'], normalize="index")
    cont_tab = cont_tab.loc[:, cont_tab.sum(axis=0) > 0.1] 

    all_cluster = adata_to_use.obs[Clusters_to_use].unique().astype(int)
    cluster_df = []
    max_corr = []
    celltype_ = []
    cellclass_ = []
    count_celltype = []
    per_celltype = []
    total_cells = len(adata_to_use)

    for all in all_cluster:
        # print(all)
        coor_temp = cont_tab_sub.T[all].sort_values(ascending = False)[0]
        max_corr.append(coor_temp)
        celltype_temp = cont_tab_sub.T[all].sort_values(ascending = False).index[0]
        celltype_.append(celltype_temp)
        cellclass_temp = cont_tab.T[all].sort_values(ascending = False).index[0]
        cellclass_.append(cellclass_temp)
        count_temp = adata_to_use.obs[adata_to_use.obs[Clusters_to_use] == all].groupby(Clusters_to_use)["cell_id"].count().values[0]
        count_celltype.append(count_temp)
        perc_temp = count_temp / total_cells * 100
        per_celltype.append(perc_temp)

    pd.set_option('display.max_rows', 250)

    d ={"Correlation":max_corr,"Celltype":celltype_,"Cell Class":cellclass_, "Cell Count":count_celltype, "Percentage": per_celltype}
    cluster_df = pd.DataFrame(data = d)

    if sort_order != 'None':
        cluster_df = cluster_df.sort_values(by=sort_order, ascending=sort_ascend)

    return cluster_df

In [ ]:
adata2.obs[adata2.obs['cell_type_final']=='VMH Glut'].groupby('sample')['cell_id'].count()

In [ ]:
dfs[59].sort_values(by='logfoldchanges')

In [ ]:
for d in range(0, len(dfs)):
    dat = dfs[d]
    dat = dat[dat['group'] == 'APP'].sort_values(by = 'logfoldchanges')
    x1 = range(0,len(dat))
    plt.scatter(x=x1, y =dat['logfoldchanges'], s =3, label = all_celltype[d])

In [ ]:
del dfs[59]

In [ ]:
### Only Keep APP values (mirror of WT)
for d in range(0,len(dfs)):
    dfs[d] = dfs[d][dfs[d]['group']=='APP']

In [ ]:
all_logfold = pd.DataFrame()
for d in range(0, len(dfs)):
    dat = dfs[d]
    log_pos = dat[dat['logfoldchanges']>0]['logfoldchanges'].count()
    log_neg = -1 * dat[dat['logfoldchanges']<0]['logfoldchanges'].count()
    temp = pd.DataFrame({'Celltype' : [all_celltype[d]],
                            'Positive':[log_pos],
                            'Negative':[log_neg]})
    all_logfold= all_logfold.append(temp, ignore_index = True)
all_logfold=all_logfold.sort_values(by='Positive',ascending = False )


In [ ]:
all_class = all_logfold['Cellclass'].unique()

fig, ax = plt.subplots(1, 1, figsize=(5,5), sharey=True)
plt.style.use('classic')
# for cla_ in range(0,len(all_class)):

temp_df = all_logfold[all_logfold['Cellclass']==all_class[2]]
temp_df.plot.bar(x='Celltype', y='Positive',
                  ax=ax,
                    color = 'darkorange', alpha = 0.7)
temp_df.plot.bar(x='Celltype', y='Negative',
                  ax=ax,
                color = 'darkgreen', alpha = 0.7)
ax.grid(False)
ax.axhline(y=0, color  = 'black')
# ax[0].set_xticklabels(g.get_xticklabels(), rotation=90)

# plt.savefig(f'plot/{name_dir}/logchange_G.svg')

In [ ]:
class_dict = {
'CA1 ProS Glut':'Neuron',
'Tanycyte':'Epithelial',
'MM Glut':'Neuron',
'CA2 FC IG Glut':'Neuron',
'SUB ProS Glut':'Neuron',
'ZI Gaba':'Neuron',
'VMH Glut':'Neuron',
'PVT Glut':'Neuron',
'CA3 Glut':'Neuron',
'HY Glut':'Neuron',
'SN Dopa':'Neuron',
'PRT Gaba':'Neuron',
'ARH GABA':'Neuron',
'SC Gaba':'Neuron',
'LA Glut':'Neuron',
'L23 RSP Glut':'Neuron',
'SC Glut':'Neuron',
'MH Glut':'Neuron',
'RT ZI GABA':'Neuron',
'PF Glut':'Neuron',
'LGv Gaba':'Neuron',
'DG Glut':'Neuron',
'TH Glut':'Neuron',
'LH Glut':'Neuron',
'Pvalb Gaba':'Neuron',
'MB Glut':'Neuron',
'APN Glut':'Neuron',
'L23 CTX Glut':'Neuron',
'Vip Gaba':'Neuron',
'MEA Glut':'Neuron',
'Endothelial':'Epithelial',
'AV Glut':'Neuron',
'Lamp5 Gaba':'Neuron',
'CHOR':'Epithelial',
'HY GABA':'Neuron',
'SN Gaba':'Neuron',
'L4 CTX Glut':'Neuron',
'Sst Gaba':'Neuron',
'Microglia':'Glia',
'L6b CTX Glut':'Neuron',
'Pericyte':'Epithelial',
'L23 PIR ENTl Glut':'Neuron',
'Astro TE':'Glia',
'L6 CTX Glut':'Neuron',
'L5 CTX Glut':'Neuron',
'SCH Gaba':'Neuron',
'STRv PAL Gaba':'Neuron',
'Ependymal':'Epithelial',
'OPC':'Glia',
'NLOT Glut':'Neuron',
'PAG Glut':'Neuron',
'MEA Gaba':'Neuron',
'Oligodendrocyte':'Glia',
'Pineal Glut':'Neuron',
'LHA Glut':'Neuron',
'Astro NT':'Glia',
'BST Glut':'Neuron',
'VLMC':'Epithelial',
'LSX Gaba':'Neuron',
'STR D1D2 Gaba':'Neuron',
'BST Gaba':'Neuron',
'STR Gaba':'Neuron',
      }

all_logfold['Cellclass'] = all_logfold['Celltype'].map(class_dict)

In [ ]:
adata.obs[adata.obs['sample']=='circa2-ZT21'].groupby('cell_type_final')['cell_id'].count()

### Gene expression plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_parquet('../notebook/csv/circa/circa_norm_combined.parquet')

df = df[df['run']=='circa4']


In [ ]:
gene_df = df.iloc[:, :5006]
d = gene_df.groupby(df['cell_type_final']).mean()

d['overzero'] = (d>=0.005).sum(axis=1)

# valid_cells = ["ABC", "AHN RCH LHA Glut", "AMY Gaba", "AMY Glut", "Astro TE", "CA1 ProS Glut",
#   "CA2 FC IG Glut", "CA3 Glut", "Choroid", "CLA EPd CTX Glut", "DG Glut", "Endothelial",
#   "Ependymal", "L2 3 IT CTX Glut", "L2 3 IT PIR ENTl Glut", "L2 3 IT RSP Glut",
#   "L4 5 IT CTX Glut", "L4 RSP ACA Glut", "L5 ET CTX Glut", "L5 NP CTX Glut",
#   "L6 CT CTX Glut", "L6 IT CTX Glut", "L6b CTX Glut", "Lamp5 Gaba", "LH Glut", "MH Glut",
#   "Microglia", "OB STR CTX Inh IMN", "Oligodendrocyte", "OPC", "Pericyte", "Pvalb Gaba",
#   "RT ZI Gaba", "Sst Gaba", "STR D1D2 Gaba", "STR PAL Gaba", "TH Glut", "Vip Gaba", 
#   "VLMC", "VMH Glut"]

# cycling = np.array([443,124,922,847,1428,826,271,498,321,674,853,1207,492,1694,1521,301,1752,267,917,455,1130,289,605,453,223,302,1002,169,1445,
          #  793,674,875,367,788,1141,536,1160,492,548,990])

valid_cells = ["ABC", "AHN RCH LHA Glut", "AMY Glut", "Astro TE", "Choroid",
      "CLA EPd CTX Glut", "Endothelial", "Ependymal", "L2 3 IT PIR ENTl Glut", "L4 5 IT CTX Glut",
      "L5 ET CTX Glut", "L5 NP CTX Glut", "L6 CT CTX Glut", "L6 IT CTX Glut", "L6b CTX Glut",
      "Lamp5 Gaba", "Microglia", "OB STR CTX Inh IMN", "Oligodendrocyte", "OPC",
      "PAL STR Gaba Chol", "Pericyte", "Pvalb Gaba", "PVH SO PVa Glut", "RT ZI Gaba", "SCH Gaba",
      "Sst Gaba", "STR D1D2 Gaba", "STR PAL Gaba", "TH Glut", "TRS BAC Glut", "Vip Gaba", "VLMC"]

cycling = [362,673,318,934,519,261,846,544,427,742,266,195,588,1122,244,155,486,311,736,380,159,555,302,182,288,337,257,859,278,405,101,186,458,]



d = d.filter(valid_cells, axis=0)

celltype = np.array(d.index) 
g_expressed = np.array(d['overzero'])
len(g_expressed), len(celltype),len(cycling)

df_cycle = pd.DataFrame(data={"Celltype": celltype,"Expressed": g_expressed - cycling,"Cycling": np.array(cycling)})



In [ ]:
celltype = df_cycle['Celltype']

genes = {
    "Expressed": df_cycle['Expressed'],
    "Cycling": df_cycle['Cycling'],
}
width = 0.75

fig, ax = plt.subplots(figsize=(10,4))
bottom = np.zeros(len(celltype))

for boolean, genes in genes.items():
    p = ax.bar(celltype, genes, width, label=boolean, bottom=bottom)
    bottom += genes

ax.tick_params(axis = 'x', rotation=90,
                top=True, labeltop=True,
                bottom = False, labelbottom=False
                )
ax.set_ylabel('# gene')
ax.legend(bbox_to_anchor=(1.1, 1.0), loc="upper left")

# fig.savefig('Gallery/Cycling_genes_order_circa4.svg')

### Circascore plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_parquet('../notebook/csv/circa/circa_norm_combined.parquet')

df = df[df['run']=='circa2']
df2 = df[df['run']=='circa4']

In [ ]:
import numpy as np

gene_df = df['circascore'].astype(int)
d = gene_df.groupby(df['cell_type_final']).mean()
e = gene_df.groupby(df['cell_type_final']).std()
g = pd.DataFrame({"Average": d, "Error":e}, index = d.index)

valid_cells = ["ABC", "AHN RCH LHA Glut", "AMY Gaba", "AMY Glut", "Astro TE", "CA1 ProS Glut",
  "CA2 FC IG Glut", "CA3 Glut", "Choroid", "CLA EPd CTX Glut", "DG Glut", "Endothelial",
  "Ependymal", "L2 3 IT CTX Glut", "L2 3 IT PIR ENTl Glut", "L2 3 IT RSP Glut",
  "L4 5 IT CTX Glut", "L4 RSP ACA Glut", "L5 ET CTX Glut", "L5 NP CTX Glut",
  "L6 CT CTX Glut", "L6 IT CTX Glut", "L6b CTX Glut", "Lamp5 Gaba", "LH Glut", "MH Glut",
  "Microglia", "OB STR CTX Inh IMN", "Oligodendrocyte", "OPC", "Pericyte", "Pvalb Gaba",
  "RT ZI Gaba", "Sst Gaba", "STR D1D2 Gaba", "STR PAL Gaba", "TH Glut", "Vip Gaba", 
  "VLMC", "VMH Glut"]
cycling = np.array([443,124,922,847,1428,826,271,498,321,674,853,1207,492,1694,1521,301,1752,267,917,455,1130,289,605,453,223,302,1002,169,1445,
           793,674,875,367,788,1141,536,1160,492,548,990])

g = g.filter(valid_cells, axis=0)
g['Cycling'] = cycling

g.sort_values('Average', ascending = False, inplace=True)


In [ ]:
plt.scatter(x=g['Average'], y=g['Cycling'])
plt.xlim(left=0)
plt.ylim(bottom=0)

from scipy import stats
stat_ = round(stats.pearsonr(x=g['Average'], y = g['Cycling'])[0],3)

plt.title(f'SCC = {stat_}')
plt.xlabel('Average circascore')
plt.ylabel('Nb of cycling genes')

In [ ]:
plt.figure(figsize=(10,4))
plt.bar(x=g.index, height=g['Average'])
plt.errorbar(x=g.index, y=g['Average'], yerr=g['Error'], fmt="none", color="black")
plt.xticks(rotation=90)
plt.ylabel('Average circascore')
plt.show()

In [ ]:
import seaborn as sns
sns.histplot(gene_df,
             stat = 'proportion',
             binwidth=1)

In [ ]:
unique_celltypeC2 = set(adata.obs[(adata.obs['run']=='circa2') & (adata.obs['cell_type_final'].value_counts() > 100)]['cell_type_final'].unique())
unique_celltypeC4 = set(adata.obs[adata.obs['run']=='circa4']['cell_type_final'].unique())

common_celltypes = unique_celltypeC2.intersection(unique_celltypeC4)
print(len(common_celltypes),common_celltypes)

In [ ]:
celltypes = adata.obs[adata.obs['run']=='circa2']['cell_type_final'].unique()
cellC2 = set([cell for cell in celltypes if adata.obs[(adata.obs['run']=='circa2') & (adata.obs['cell_type_final']== cell)]['cell_type_final'].count() > 100])

celltypes4 = adata.obs[adata.obs['run']=='circa4']['cell_type_final'].unique()
cellC4 = set([cell for cell in celltypes4 if adata.obs[(adata.obs['run']=='circa4') & (adata.obs['cell_type_final']== cell)]['cell_type_final'].count() > 100])

In [ ]:
common_celltypes = cellC4.intersection(cellC2)
print(len(common_celltypes),common_celltypes)

In [ ]:
x1 = adata.obs[(adata.obs['run'] == 'circa2') & (adata.obs['circascore']==0)]['cell_type_final'].value_counts().sort_index()
x2 = adata.obs['cell_type_final'].value_counts().sort_index()
x3 = x1 / x2 * 100
x3 = x3.filter(valid_cells, axis = 0)
x3.sort_values(inplace=True, ascending = False)

plt.figure(figsize=(10,4))
plt.bar(x=x3.index, height=x3)
plt.tick_params(rotation = 90)
plt.ylabel('% of cells without clock genes')
plt.ylim(0,20)
plt.xlim(-0.5,40)

In [ ]:
adata.obs['cell_class'].unique()

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (15,5))
axs = axs.flatten()

for x,class_cell in enumerate(adata.obs['cell_class'].unique()):
    for cell in adata.obs[adata.obs['cell_class']==class_cell]['cell_type_final'].unique():
        x1= adata.obs[adata.obs['cell_type_final']==cell]['circascore'].value_counts().sort_index()
        x1 = x1 / adata.obs[adata.obs['cell_type_final']==cell]['cell_id'].count() * 100
        axs[x].plot(x1, label = cell)
        axs[x].legend(bbox_to_anchor=(1.0, 1.0), loc="lower center")

In [ ]:
region_ = "CTX"
cell = 'Sst Gaba'
x1= adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_type_final']==cell) & (adata.obs['cell_class']=="Neuronal")]['circascore'].value_counts().sort_index()

In [ ]:
x1.max()

In [ ]:
fig, axs = plt.subplots(4,4, figsize = (15,15))
axs = axs.flatten()

for x,region_ in enumerate(adata.obs['region_automap_name'].unique()):
    x3 = [0]
    for cell in adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_class']=="Neuronal")]['cell_type_final'].unique():
        x1= adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_type_final']==cell) & (adata.obs['cell_class']=="Neuronal")]['circascore'].value_counts().sort_index()
        x2 = adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_type_final']==cell) & (adata.obs['cell_class']=="Neuronal")]['cell_id'].count()
        if x2.sum() > 100:
            x1 = x1 / adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_class']=="Neuronal")]['cell_id'].count() * 100
            x1_max = x1.max()
            x3.insert(0, x1_max)
            axs[x].plot(x1, label = f'{cell}_{x1.idxmax()}')
            # axs[x].legend(loc="best")
    axs[x].set_title(region_)
    axs[x].set_xlim(0,11)
    axs[x].set_ylim(0,max(x3)*1.25)


In [ ]:
fig, axs = plt.subplots(4,4, figsize = (15,15))
axs = axs.flatten()

color_dict = {'Astro TE': 'black',
              'Oligodendrocyte':'red',
              'Microglia':"green",
              'OPC':'blue'}

for x,region_ in enumerate(adata.obs['region_automap_name'].unique()):
    x3 = [0]
    for cell in adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_class']=="Glial")]['cell_type_final'].unique():
        x1= adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_type_final']==cell) & (adata.obs['cell_class']=="Glial")]['circascore'].value_counts().sort_index()
        x2 = adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_type_final']==cell) & (adata.obs['cell_class']=="Glial")]['cell_id'].count()
        if x2.sum() > 100:
            x1 = x1 / adata.obs[(adata.obs['region_automap_name']==region_) & (adata.obs['cell_class']=="Glial")]['cell_id'].count() * 100
            x1_max = x1.max()
            x3.insert(0, x1_max)
            axs[x].plot(x1, c = color_dict[cell], label = f'{cell}_{x1.idxmax()}')
            axs[x].legend(loc="best")
    axs[x].set_title(region_)
    axs[x].set_xlim(0,11)
    axs[x].set_ylim(0,max(x3)*1.25)


In [ ]:
fig, axs = plt.subplots(4,4, figsize = (15,15))
axs = axs.flatten()

for x,region_ in enumerate(adata.obs['region_automap_name'].unique()):
    # for cell in adata.obs[adata.obs['region_automap_name']==region_]['cell_type_final'].unique():
        x1= adata.obs[(adata.obs['region_automap_name'] == region_) & (adata.obs['cell_type_final']=='OPC')]['circascore'].value_counts().sort_index()
        x1 = x1 / adata.obs[(adata.obs['region_automap_name'] == region_) & (adata.obs['cell_type_final']=='Pericyte')]['cell_id'].count() * 100
        axs[x].plot(x1, label = region_)
        axs[x].legend(bbox_to_anchor=(1.0, 1.0), loc="best")
        axs[x].set_xlim(0,11)
        axs[x].set_ylim(0,50)

In [ ]:
df = pd.read_parquet('../notebook/csv/circa/circa_norm_combined.parquet')

In [ ]:
import matplotlib.pyplot as plt

x = df[df['cell_type_final']=='STR D1D2 Gaba'].groupby('sample')['Ndn'].mean()

plt.scatter(x = x.index, y=x)
plt.tick_params(rotation=90)
plt.vlines(x=(5.5,11.5), ymin=0,ymax=0.5)
plt.xlim(5.5,)
plt.title('Ndn expression in \'STR D1D2 Gaba\'')
plt.ylabel('Normalized expression')

In [ ]:
x1 = df[df['Clock'] > 0].groupby('cell_type_final')['Clock'].count().sort_index() / df.groupby('cell_type_final')['Clock'].count().sort_index()
x2 = df[df['Npas2'] > 0].groupby('cell_type_final')['Npas2'].count().sort_index() / df.groupby('cell_type_final')['Npas2'].count().sort_index()

In [ ]:
x1.max(),x2.max()

In [ ]:
x_2 = x1.index
d = {"Clock": x1.values,
     "Npas2": x2.values
     }

df = pd.DataFrame(data=d, index = x_2)
df = df.sort_values(by = 'Clock', ascending=False, ignore_index=True)


plt.figure(figsize=(15,5), layout='constrained')

for x_, d_ in enumerate(df.columns):
    plt.bar(x=x_2, height=df[d_], label = d_, alpha=0.5)
plt.legend(loc = 'best')
plt.ylabel('Portion of cells expressing')
plt.tick_params(axis = "x", rotation=90)